In [0]:
dbutils.widgets.text('source', '')

In [0]:
source_value = dbutils.widgets.get('source')

In [0]:
%sql
-- CREATE SCHEMA workspace.raw;
-- CREATE SCHEMA workspace.bronze;
-- CREATE SCHEMA workspace.silver;
-- CREATE SCHEMA workspace.gold;

In [0]:
%sql
-- CREATE VOLUME workspace.bronze.volume;
-- CREATE VOLUME workspace.silver.volume;
-- CREATE VOLUME workspace.gold.volume;

In [0]:
# dbutils.fs.mkdirs("/Volumes/workspace/raw/raw_data/customers")
# dbutils.fs.mkdirs("/Volumes/workspace/raw/raw_data/airports")
# dbutils.fs.mkdirs("/Volumes/workspace/raw/raw_data/bookings")
# dbutils.fs.mkdirs("/Volumes/workspace/raw/raw_data/flights")

In [0]:
df = spark.readStream.format('cloudFiles').options(
    **{
        'cloudFiles.format':'csv',
        'cloudFiles.schemaLocation':f'/Volumes/workspace/bronze/volume/{source_value}/checkpoint',
        'cloudFiles.schemaEvolutionMode':'rescue'
    }
).load(f'/Volumes/workspace/raw/raw_data/{source_value}/')
    

In [0]:
df.writeStream.format('delta').options(
    **{
        'outputMode':'append',
        'checkpointLocation': f'/Volumes/workspace/bronze/volume/{source_value}/checkpoint',
        'path': f'/Volumes/workspace/bronze/volume/{source_value}/data'
    }
).trigger(once=True).start()

In [0]:
%sql
SELECT * FROM delta.`/Volumes/workspace/bronze/volume/airports/data`